# Demo caching

This notebook shows how caching of daily results is organised. First we show the low-level approach, then a high-level function is used.

## Low-level approach

In [ ]:
import pandas as pd
from opengrid.library import misc
from opengrid.library import houseprint
from opengrid.library import caching
import charts
hp = houseprint.Houseprint()

We demonstrate the caching for the minimal daily water consumption (should be close to zero unless there is a water leak).  We create a cache object by specifying what we like to store and retrieve through this object.  The cached data is saved as a single csv per sensor in a folder specified in the opengrid.cfg.  Add the path to a folder where you want these csv-files to be stored as follows to your opengrid.cfg

    [data]
    folder: path_to_folder

In [ ]:
cache_water = caching.Cache(variable='water_daily_min')
df_cache = cache_water.get(sensors=hp.get_sensors(sensortype='water'))
charts.plot(df_cache.ix[-8:], stock=True, show='inline')

If this is the first time you run this demo, no cached data will be found, and you get an empty graph. 

Let's store some results in this cache.  We start from the water consumption of last week.

In [ ]:
hp.sync_tmpos()

In [ ]:
start = pd.Timestamp('now') - pd.Timedelta(weeks=1)
df_water = hp.get_data(sensortype='water', head=start, )
df_water.info()

We use the method *daily_min()* from the analysis module to obtain a dataframe with daily minima for each sensor.

In [ ]:
daily_min = analysis.DailyAgg(df_water, agg='min').result
daily_min.info()

In [ ]:
daily_min

In [ ]:
cache_water.update(daily_min)

Now we can get the daily water minima from the cache directly. Pass a *start* or *end* date to limit the returned dataframe.

In [ ]:
sensors = hp.get_sensors(sensortype='water') # sensor objects
charts.plot(cache_water.get(sensors=sensors, start=start, end=None), show='inline', stock=True)

## A high-level cache function

The caching of daily results is very similar for all kinds of results.  Therefore, a high-level function is defined that can be parametrised to cache a lot of different things.

In [1]:
import pandas as pd
from opengrid.library import misc
from opengrid.library import houseprint
from opengrid.library import caching
from opengrid.library import analysis
import charts   

Server running in the folder /mnt/data/work/opengrid/code/og/notebooks at 127.0.0.1:53046


In [2]:
hp = houseprint.Houseprint()
#hp.sync_tmpos()

Opening connection to Houseprint sheet
Opening spreadsheets
Parsing spreadsheets
24 Sites created
24 Devices created
75 sensors created
Houseprint parsing complete


In [4]:
sensors = hp.get_sensors(sensortype='water')
caching.cache_results(hp=hp, sensors=sensors, resultname='water_daily_min', AnalysisClass=analysis.DailyAgg, agg='min')

Cache object created for variable: water_daily_min
Using tmpo database from /mnt/data/work/opengrid/data/.tmpo/tmpo.sqlite3


True

In [5]:
cache = caching.Cache('water_daily_min')
daily_min = cache.get(sensors = sensors, start = '20151201')
charts.plot(daily_min, stock=True, show='inline')

Cache object created for variable: water_daily_min


In [6]:
daily_min

,661684d35cd07dfc657a60b844860785,b28509eb97137e723995838c393d49df,2923b75daf93e539e37ce5177c0008c5,a8054ca7865584e69a7c946aeea1e13c,1a5c55ec176d24483572aca420ac59b5,c57f45b21e2ec21ab427f172478e1ca4,a926bc966f178fc5d507a569a5bfc3d7,a519ce042e8748570d3a91cfca16a747
2016-01-28 00:00:00+00:00,0.001289,0.011173,0.002898,0.004445,0.000611,0.001116,0.017889,NaN
2016-01-29 00:00:00+00:00,0.001104,0.018462,0.000905,0.001359,0.000445,0.001050,0.011009,NaN
2016-01-30 00:00:00+00:00,0.001014,0.009622,0.000801,0.000948,0.000445,0.001012,0.008227,NaN
2016-01-31 00:00:00+00:00,0.001014,0.012959,0.000801,0.000948,0.000487,0.001012,0.012348,NaN
2016-02-01 00:00:00+00:00,0.000952,0.015480,0.001029,0.001626,0.001924,0.001192,0.033003,NaN
2016-02-02 00:00:00+00:00,0.001130,0.001405,0.001029,0.001652,0.000829,0.000874,0.023933,NaN
2016-02-03 00:00:00+00:00,0.001130,0.001405,0.001045,0.001051,0.000829,0.001300,0.023337,NaN
2016-02-04 00:00:00+00:00,0.005516,0.001923,0.012717,0.063025,NaN,0.012953,0.023962,NaN
2016-02-18 00:00:00+00:00,0.001344,0.023059,0.008603,NaN,0.032258,NaN,0.050891,NaN
2016-02-19 00:00:00+00:00,0.001273,0.012594,0.001807,0.000978,0.000572,0.000959,0.014606,NaN


In [7]:
daily_min.index

DatetimeIndex(['2016-01-28', '2016-01-29', '2016-01-30', '2016-01-31',
               '2016-02-01', '2016-02-02', '2016-02-03', '2016-02-04',
               '2016-02-18', '2016-02-19', '2016-02-20', '2016-02-21',
               '2016-02-22', '2016-02-23', '2016-02-24', '2016-02-25',
               '2016-02-26', '2016-02-27', '2016-02-28', '2016-02-29',
               '2016-03-01', '2016-03-02', '2016-03-03', '2016-03-04',
               '2016-03-05', '2016-03-06', '2016-03-07', '2016-03-08',
               '2016-03-09', '2016-03-10', '2016-03-11', '2016-03-12',
               '2016-03-13', '2016-03-14', '2016-03-15', '2016-03-16',
               '2016-03-17', '2016-03-18', '2016-03-19', '2016-03-20',
               '2016-03-21', '2016-03-22', '2016-03-23', '2016-03-24',
               '2016-03-25', '2016-03-26', '2016-03-27', '2016-03-28',
               '2016-03-29', '2016-03-30', '2016-03-31', '2016-04-01',
               '2016-04-02', '2016-04-03', '2016-04-04', '2016-04-05',
      